In [1]:
import os
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role

# role = get_execution_role()
# print(role)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/user/Library/Application Support/sagemaker/config.yaml


In [2]:
pytorch_model = PyTorchModel(
    name='embeddings-model',
    model_data='s3://llama-weights-us/labse-model/model.tar.gz',
    role=os.environ.get('SAGEMAKER_ROLE'),
    framework_version="2.0.0",
    py_version="py310",
    source_dir="code",
    entry_point="inference.py",
    env={"MODEL_NAME": "all-MiniLM-L12-v2"},
)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/user/Library/Application Support/sagemaker/config.yaml


In [3]:
transformer = pytorch_model.transformer(
    instance_count=1, 
    instance_type="ml.g4dn.xlarge",
    accept='text/csv'
    )


# we can run it directly
# transformer.transform(
#     "s3://llama-weights-us/input_data/",
#     content_type="text/csv",
#     split_type="Line",
#     wait=True,

# )

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/user/Library/Application Support/sagemaker/config.yaml


Using already existing model: embeddings-model


In [26]:
from sagemaker.transformer import Transformer

transformer = Transformer(
        model_name='embeddings-model',
        output_path="s3://llama-weights-us/input_data/",
        instance_count=1,
        instance_type="ml.g4dn.xlarge",
        accept='text/csv',
        assemble_with='Line',
        strategy='MultiRecord',
        max_payload=70,
        env={
            'BATCH_SIZE': str(64),
            'STRATEGY': 'MultiRecord',
            'MODEL_NAME': "all-MiniLM-L12-v2"}
    )

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/user/Library/Application Support/sagemaker/config.yaml


In [27]:

transformer.transform(
    "s3://llama-weights-us/input_data/",
    content_type="text/csv",
    split_type="Line",
    wait=True,

)


INFO:sagemaker:Creating transform job with name: pytorch-inference-2023-11-24-16-15-47-877


.................................................Collecting datasets==2.15.0 (from -r /opt/ml/model/code/requirements.txt (line 1))
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 46.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 53.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [31]:
import pandas as pd 
df = pd.read_csv('s3://llama-weights-us/input_data/texts.csv.out', header=None)
df

,0,1,2,3,4,5,6,7,8,9,...,375,376,377,378,379,380,381,382,383,384
0,ישראל חי עם,-0.028912,0.114269,-0.114297,-0.022898,-0.009962,-0.046699,0.148093,-0.026738,0.058141,...,-0.011244,0.050298,-0.031433,0.059069,0.039949,0.012332,0.011578,0.028020,-0.042433,-0.061470
1,עם ישראל חי,-0.024890,0.123166,-0.112473,-0.032596,-0.002534,-0.045722,0.158048,-0.020617,0.050401,...,0.001951,0.040327,-0.031516,0.049747,0.031777,0.012196,0.004450,0.040684,-0.049453,-0.061965
2,תנו לצהל לנצח,-0.060326,0.103238,-0.001080,0.061596,0.012187,-0.081065,0.112682,-0.028075,0.074386,...,0.014554,-0.019529,0.043713,0.017257,-0.066258,0.008436,-0.038106,-0.008491,-0.036781,-0.050527
